In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


In [25]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("../files/chapter_three.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()


In [26]:
# 프롬프트 템플릿 정의
template = """다음 정보를 사용하여 질문에 답변하세요:

{context}

질문: {question}
이전 대화:
{chat_history}

답변:"""

PROMPT = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template=template
)

In [27]:
# 메모리 초기화
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [28]:
# 체인 수동 구현
def retrieve_docs(query):
    return retriever.get_relevant_documents(query)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def generate_answer(input_dict):
    context = input_dict["context"]
    question = input_dict["question"]
    chat_history = input_dict["chat_history"]
    
    prompt = PROMPT.format(context=context, question=question, chat_history=chat_history)
    return llm.invoke(prompt)

In [31]:
chain = (
    {
        "context": RunnableLambda(retrieve_docs) | RunnableLambda(format_docs),
        "question": RunnablePassthrough(),
        "chat_history": lambda x: memory.load_memory_variables({})["chat_history"]
    }
    | RunnableLambda(generate_answer)
)

In [32]:
# 질문 및 답변 함수
def ask_question(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result})
    return result

In [36]:
# 질문하기
questions = [
    "Aaronson 은 유죄인가요?",
    "그가 테이블에 어떤 메시지를 썼나요?",
    "Julia 는 누구인가요?"
]

# chain.invoke("Aaronson 은 유죄인가요?")
ask_question("Aaronson 은 유죄인가요?")



ValidationError: 21 validation errors for AIMessage
content.str
  Input should be a valid string [type=string_type, input_value=AIMessage(content='제공...ls': {'cache_read': 0}}), input_type=AIMessage]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
content.list[union[str,dict[any,any]]].0.str
  Input should be a valid string [type=string_type, input_value=('content', '제공된 ...전혀 없습니다.\n'), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
content.list[union[str,dict[any,any]]].0.dict[any,any]
  Input should be a valid dictionary [type=dict_type, input_value=('content', '제공된 ...전혀 없습니다.\n'), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.9/v/dict_type
content.list[union[str,dict[any,any]]].1.str
  Input should be a valid string [type=string_type, input_value=('additional_kwargs', {}), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.9/v/string